In [1]:
import xgboost

In [2]:
import pandas as pd

In [3]:
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [5]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]
dependent=dataset[['charges']]

In [11]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
param_grid={"n_estimators":[100,500,1000], 'max_depth':[3,4,5], 'alpha':[1,2,3], 'learning_rate':[0.03,0.05,0.07],
            'subsample':[0.6,0.8,1.0],'colsample_bytree':[0.6,0.7,0.8]}
grid=GridSearchCV(XGBRegressor(), param_grid, refit=True, verbose=3, n_jobs=-1)
grid.fit(independent,dependent)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits


GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_bi...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             n_jobs=-1,
             param_grid={'alpha': [1, 2, 3],
                         'colsample_bytree': [0.6, 0.7, 0.8],
                         'learning_rate': [0.03, 0.05, 0.07],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 500, 1000],
                         'subsample': [0.6, 0.8, 1.0]},
             verbose=3)

In [13]:
re=grid.cv_results_
print("The R_score value for best parameter {}:".format(grid.best_params_))

The R_score value for best parameter {'alpha': 3, 'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}:


In [15]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_colsample_bytree,param_learning_rate,param_max_depth,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.040266,0.008743,0.006075,0.002210,1,0.6,0.03,3,100,0.6,"{'alpha': 1, 'colsample_bytree': 0.6, 'learnin...",0.837146,0.783972,0.820077,0.813522,0.822343,0.815412,0.017517,582
1,0.031321,0.000987,0.005267,0.000697,1,0.6,0.03,3,100,0.8,"{'alpha': 1, 'colsample_bytree': 0.6, 'learnin...",0.837589,0.787369,0.820369,0.811527,0.824481,0.816267,0.016712,576
2,0.025922,0.002768,0.005239,0.001748,1,0.6,0.03,3,100,1.0,"{'alpha': 1, 'colsample_bytree': 0.6, 'learnin...",0.838083,0.788146,0.822485,0.810322,0.824522,0.816712,0.016780,565
3,0.120916,0.002918,0.004861,0.000422,1,0.6,0.03,3,500,0.6,"{'alpha': 1, 'colsample_bytree': 0.6, 'learnin...",0.873842,0.798131,0.880820,0.847267,0.856186,0.851250,0.029143,86
4,0.117414,0.002071,0.004857,0.000733,1,0.6,0.03,3,500,0.8,"{'alpha': 1, 'colsample_bytree': 0.6, 'learnin...",0.873584,0.800754,0.880696,0.846859,0.858114,0.852001,0.028211,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,0.229632,0.014732,0.008298,0.001746,3,0.8,0.07,5,500,0.8,"{'alpha': 3, 'colsample_bytree': 0.8, 'learnin...",0.837380,0.744441,0.835124,0.823723,0.824387,0.813011,0.034724,601
725,0.164637,0.009811,0.007019,0.000607,3,0.8,0.07,5,500,1.0,"{'alpha': 3, 'colsample_bytree': 0.8, 'learnin...",0.834370,0.751710,0.844902,0.814116,0.829927,0.815005,0.033162,595
726,0.402980,0.008583,0.009437,0.000953,3,0.8,0.07,5,1000,0.6,"{'alpha': 3, 'colsample_bytree': 0.8, 'learnin...",0.810678,0.723645,0.811591,0.812145,0.807160,0.793044,0.034743,718
727,0.420173,0.024345,0.011590,0.000614,3,0.8,0.07,5,1000,0.8,"{'alpha': 3, 'colsample_bytree': 0.8, 'learnin...",0.820599,0.723773,0.811383,0.814952,0.809659,0.796073,0.036343,710


In [23]:
#to view full rows of table:
pd.set_option('display.max_rows', None)
print(table)

     mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0         0.040266      0.008743         0.006075        0.002210   
1         0.031321      0.000987         0.005267        0.000697   
2         0.025922      0.002768         0.005239        0.001748   
3         0.120916      0.002918         0.004861        0.000422   
4         0.117414      0.002071         0.004857        0.000733   
5         0.091189      0.006439         0.005984        0.000567   
6         0.230559      0.006927         0.006771        0.001089   
7         0.224477      0.003346         0.006282        0.000403   
8         0.174603      0.005266         0.006813        0.001045   
9         0.037393      0.002586         0.004055        0.000044   
10        0.034903      0.001718         0.004863        0.001014   
11        0.033581      0.002422         0.004541        0.000791   
12        0.139656      0.002673         0.006474        0.000379   
13        0.140688      0.003764  

In [19]:
age_input=float(input("Age:"))
bmi_input=float(input("BMI:"))
children_input=float(input("Children:"))
sex_male_input=int(input("Sex Male 0 or 1:"))
smoker_yes_input=int(input("Smoker Yes 0 or 1:"))

Age: 26
BMI: 23
Children: 2
Sex Male 0 or 1: 0
Smoker Yes 0 or 1: 1


In [21]:
Future_Prediction=grid.predict([[age_input,bmi_input,children_input,sex_male_input,smoker_yes_input]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[18623.912]
